Este notebook es una versión de los notebook de [kkiller](https://www.kaggle.com/kneroma/inference-resnest-rfcx-audio-detection) y de [Tarek Hamdi](https://www.kaggle.com/hamditarek/rainforest-connection-analysis-using-librosa).

Mi aportación ha sido la traducción y la explicación en español de que es lo que se consigue con este código, utilizando los paquetes de librosa y pytorch principalmente.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn

import IPython.display as ipd
import librosa
import librosa.display
import soundfile as sf

### Usaremos los siguientes paquetes de análisis de audio:
## Librosa
Es un módulo de Python que analiza señales de audio en general. Incluye lo necesario para crear un sistema MIR (Music Information Retrieval), idóneo para el desarrollo de este análisis. Podemos encontrar la documentación [aquí](https://librosa.org/librosa/), con muchos ejemplos y tutoriales.

## IPython.display.Audio
Que nos permitirá reproducir el audio directamente desde el "Notebook" para facilitar la comprensión del código.

# 1. Carga del archivo de audio

In [ ]:
# De la siguiente manera cargamos el archivo de audio en el Notebook, que será una serie temporal a modo de 'array'
# con una tasa de muestreo 'sr' de 22 kHZ mono.

audio_test_1 = "../input/rfcx-species-audio-detection/train/0d25045a9.flac"

# Seleccioné como ejemplo ese audio por tener 1 especie presente

x , sr = librosa.load(audio_test_1)

print(type(x), type(sr))
print(x.shape, sr)

In [ ]:
# Si nos interesa, podemos cambiar la tasa de muestreo a 44.1 kHZ

librosa.load(audio_test_1, sr = 44100)

In [ ]:
# Para poder escuchar el audio aquí en el 'Notebook' hacemos lo siguiente:

ipd.Audio(audio_test_1)

# 2. Generación de Espectrogramas

Un Espectrograma es una manera de representar el audio de manera VISUAL. A lo largo del tiempo que dura dicho audio, se puede representar gráficamente las frecuencias que se han registrado.

El Espectograma puede mostrarse de distintas formas, ya sea como una gráfica de líneas o como un 'Heatmap'

In [ ]:
# Mostramos el Espectrograma como una gráfica de líneas.
# Tiempo vs Frecuencia

%matplotlib inline

plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

In [ ]:
# Mostramos el Espectrograma como un'Heatmap'.
# Tiempo vs Frecuencia

X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

Short Time Fourier Transform (stft()) convierte los datos en, como su nombre indica, [transformadas de Fourier de tiempo corto](https://es.wikipedia.org/wiki/Transformada_de_Fourier_de_Tiempo_Reducido), usadas para determinar el contenido en frecuencia sinusoidal y de fase en secciones locales de una señal así como sus cambios con respecto al tiempo.
STFT convierte las señales de tal manera que podemos saber la amplitud de la frecuencia dada en un momento dado y además podemos determinar la amplitud de varias frecuencias que se reproducen en un momento dado de una señal de audio.

Por otro lado, specshow() muestra el espectrograma con las siguientes características: El eje vertical muestra las frecuencias (de 0 a 10kHz), y el eje horizontal muestra el tiempo del clip. Como vemos que toda la acción tiene lugar en el fondo del espectro, podemos convertir el eje de frecuencias en uno logarítmico.

In [ ]:
# Aplicamos la escala logarítimica para centrar la atención el los sonidos a estudiar

librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

# 3. Creación de la señal de Audio

In [ ]:
sr = 22050 # Tiempo de muestreo
T = 5.0    # Segundos
t = np.linspace(0, T, int(T*sr), endpoint=False) # Variable tiempo
x = 0.5*np.sin(2*np.pi*220*t) # Onda senoidal pura a 220 Hz

# Reproduciendo el audio
ipd.Audio(x, rate=sr) # Generando un NumPy 'array'

# Guardando el audio
sf.write('tono_0d25045a9_220.wav', x, sr)

In [ ]:
# El 'spectral_centroid' devolverá un 'array' con el mismo número de 
# columnas que número de 'fotogramas' presentes en la muestra de audio

spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
spectral_centroids.shape

# Calculando la variable de tiempo para la visualización

plt.figure(figsize=(12, 4))
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)

# Normalizando el 'centroide espectral' para la visualización

def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

# Trazando el 'centroide espectral' a lo largo de la forma de onda

librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='b')

## Atenuación del espectro de frecuencias

Representa la frecuencia a la que las frecuencias altas descienden a 0. Para obtener dicha frecuencia, hay que calcular la fracción en el espectro en la cual el 85% de la potencia espectral está a bajas frecuencias.

librosa.feature.spectral_rolloff() calcula la frecuencia de caida para cada momento de la señal

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(x+0.01, sr=sr)[0]
plt.figure(figsize=(12, 4))
librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_rolloff), color='black')

## Ancho de banda del espectro de frecuencias

El Ancho de Banda del espectro es el intervalo de longitudes de onda en el que una cantidad espectral no es inferior a la mitad de su valor máximo.

In [ ]:
spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr)[0]
spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=3)[0]
spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=4)[0]
plt.figure(figsize=(15, 9))
librosa.display.waveplot(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2), color='r')
plt.plot(t, normalize(spectral_bandwidth_3), color='g')
plt.plot(t, normalize(spectral_bandwidth_4), color='y')
plt.legend(('p = 2', 'p = 3', 'p = 4'))

In [ ]:
x.shape

In [ ]:
x, sr = librosa.load(audio_test_1)

# Para centrarnos en un momento específico y ver el comportamiento de las frecuencias, podemos
# mostrar la gráfica del audio en cuestión para hacerle un zoom en el momento concreto:

plt.figure(figsize=(14, 5))
librosa.display.waveplot(x, sr=sr)

# Aplicamos el zoom, siendo 9000 y 9100 datos registrados en 'x'.

# Dado que 'x' es un "array" de 1.323.000 valores, acotando a los valores 
# mencionados, conseguimos hacer zoom a una zona en concreta del audio.

# Haciendo algunos cálculos rápidamente, si la duración total es de 60 segundos
# y en esos 60 segundos tenemos 1.323.000 valores, el zoom de 9000 a 9100 equivale
# al periodo de tiempo entre el segundo 0'4081 y el segundo 0'4126.

n0 = 9000
n1 = 9100
plt.figure(figsize=(14, 5))
plt.plot(x[n0:n1])
plt.grid()

In [ ]:
# Librosa también nos permite calcular fácilmente el número de veces
# que la onda pasa por cero en el periodo de tiempo determinado

zero_crossings = librosa.zero_crossings(x[n0:n1], pad=False)
print(sum(zero_crossings))

# 4. Mel-Frequency Cepstral Coefficients (MFCCs)

Los MFCCs son coeﬁcientes para la representación del habla basados en la percepción auditiva humana. Estos surgen de la necesidad, en el área del reconocimiento de audio automático, de extraer características de las componentes de una señal de audio que sean adecuadas para la identificación de contenido relevante, así como obviar todas aquellas que posean información poco valiosa como el ruido de fondo, emociones, volumen, tono, etc.

In [ ]:
# Para obtener una gráfica MFCC, se definen los siguientes parámetros:

fs=10  # Siendo 'fs' un escalar > 0 y que define la tasa de muestreo para el eje "y" 
mfccs = librosa.feature.mfcc(x, sr = fs)  # Se define la variable mfccs para poder sacar el gráfico, es una matriz
print(mfccs.shape)
(20, 97)

# Utilizando la libreria 'librosa' sacamos el gráfico correspondiente

plt.figure(figsize=(15, 7))
librosa.display.specshow(mfccs, sr = sr, x_axis = 'time')  # Siendo 'sr' la tasa de muestreo utilizada para determinar la escala de tiempo en el eje "x".

In [ ]:
# Otra herramienta interesante es 'chromagram', pues permite resaltar las características principales del croma

hop_length=12  # La longitud del 'salto', que también se utiliza para determinar la escala de tiempo en el eje "x"
chromagram = librosa.feature.chroma_stft(x, sr = sr, hop_length = hop_length)  # Se define la variable 'chromagram' para poder sacar el gráfico, es una matriz

# Y se obtiene el gráfico con las variables ya definidas

plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram,
                         x_axis = 'time', 
                         y_axis = 'chroma',
                         hop_length = hop_length, 
                         cmap = 'coolwarm'  #'magma', 'gray_r', 'coolwarm'
                         )

# 5. Modelado

In [ ]:
# Trabajando en Linux desde mi ordenador, la manera de instalar el paquete 'resnest' fue utilizando:

#pip install resnest

# Pero para poder subir el Notebook a kaggle, la forma de implementar resnest es la siguiente

!pip install resnest > /dev/null

In [ ]:
from pathlib import Path
import librosa as lb

import torch
from  torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm

from resnest.torch import resnest50

In [ ]:
# Se definen en primer lugar algunas variables y se cargan los datos de la competición para 
# implementarles transformaciones mediante unas funciones que se analizarán más adelante

NUM_CLASSES = 24
SR = 16_000
DURATION =  60

DATA_ROOT = Path("../input/rfcx-species-audio-detection")
TRAIN_AUDIO_ROOT = Path("../input/rfcx-species-audio-detection/train")
TEST_AUDIO_ROOT = Path("../input/rfcx-species-audio-detection/test")

In [ ]:
# Esta primera función crea 'MelSpecComputer', para generar 'melspec',
# añadiéndole como parámetros para poder utilizar dicha función:

class MelSpecComputer:
    def __init__(self, sr, n_mels, fmin, fmax):
        
        self.sr = sr  # sr: Tasa de muestreo del eje "y"
        self.n_mels = n_mels  # n_mels: Número de bandas 'Mel' a generar
        self.fmin = fmin  # fmin: Frecuencia máxima
        self.fmax = fmax  # fmax: Frecuencia mínima

    def __call__(self, y):

        melspec = lb.feature.melspectrogram(y,
                                            sr = self.sr,
                                            n_mels = self.n_mels,
                                            fmin = self.fmin,
                                            fmax = self.fmax,
                                            )

        melspec = lb.power_to_db(melspec).astype(np.float32)
        
        return melspec

In [ ]:
# Esta primera función se encarga de transformar 'X' en 'V', que contendrá
# la información necesaria dentro del 'array' para pasar de "mono" a "color"

def mono_to_color(X, eps=1e-6, mean=None, std=None):
    
    X = np.stack([X, X, X], axis=-1)

    # Estandarización
    mean = mean or X.mean()
    std = std or X.std()
    X = (X - mean) / (std + eps)

    # Normalización a [0, 255]
    _min, _max = X.min(), X.max()

    if (_max - _min) > eps:
        V = np.clip(X, _min, _max)
        V = 255 * (V - _min) / (_max - _min)
        V = V.astype(np.uint8)
    
    else:
        
        V = np.zeros_like(X, dtype=np.uint8)

    return V


# Esta función se encarga de normalizar la imagen

def normalize(image, mean=None, std=None):
    
    image = image / 255.0
    
    if mean is not None and std is not None:
        
        image = (image - mean) / std
        
    return np.moveaxis(image, 2, 0).astype(np.float32)

# Y esta determinará la longitud de los audios en función de los 
# datos dados por la competición (tmax - tmin)

def crop_or_pad(y, length, sr, is_train=True):
    
    if len(y) < length:
        
        y = np.concatenate([y, np.zeros(length - len(y))])
        
    elif len(y) > length:
        
        if not is_train:
            
            start = 0
            
        else:
            
            start = np.random.randint(len(y) - length)

        y = y[start:start + length]

    y = y.astype(np.float32, copy=False)

    return y

In [ ]:
# Una vez definidas las 3 funciones en la celda anterior, el siguiente paso es 
# completar un nuevo dataset 'RFCXDataset' utilizando las funciones anteriores
# a modo de preparación de los datos que se recogerán en el nuevo dataset

class RFCXDataset(Dataset):

    def __init__(self, 
                 data, 
                 sr, 
                 n_mels = 128, 
                 fmin = 0, 
                 fmax = None,  
                 is_train = False,
                 num_classes = NUM_CLASSES, 
                 root = None, 
                 duration = DURATION) :

        self.data = data
        
        self.sr = sr
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax or self.sr//2

        self.is_train = is_train

        self.num_classes = num_classes
        self.duration = duration
        self.audio_length = self.duration*self.sr
        
        self.root =  root or (TRAIN_AUDIO_ROOT if self.is_train else TEST_AUDIO_ROOT)

        self.wav_transfos = get_wav_transforms() if self.is_train else None

        self.mel_spec_computer = MelSpecComputer(sr=self.sr, n_mels=self.n_mels, fmin=self.fmin, fmax=self.fmax)  # Función definida en la celda anterior


    def __len__(self):
        return len(self.data)
    
    def read_index(self, idx, fill_val=1.0, offset=None, use_offset=True):
        d = self.data.iloc[idx]
        record, species = d["recording_id"], d["species_id"]
        try:
            if use_offset and (self.duration < d["duration"]+1):
                offset = offset or np.random.uniform(1, int(d["duration"]-self.duration))

            y, _ = lb.load(self.root.joinpath(record).with_suffix(".flac").as_posix(),
                           sr=self.sr, duration=self.duration, offset=offset)
            
            if self.wav_transfos is not None:
                y = self.wav_transfos(y, self.sr)
            y = crop_or_pad(y, self.audio_length, sr=self.sr)  # Función definida en la celda anterior
            t = np.zeros(self.num_classes)
            t[species] = fill_val
            
        except Exception as e:
#             print(e)
            raise ValueError()  from  e
            y = np.zeros(self.audio_length)
            t = np.zeros(self.num_classes)
        
        return y,t
            
        

    def __getitem__(self, idx):

        y, t = self.read_index(idx)
        
        
        melspec = self.mel_spec_computer(y)  # Función definida en la celda anterior
        image = mono_to_color(melspec)  # Función definida en la celda anterior
        image = normalize(image, mean=None, std=None)  # Función definida en la celda anterior

        return image, t

In [ ]:
# Función para obtener la duración de los audios

def get_duration(audio_name, root=TEST_AUDIO_ROOT):
    return lb.get_duration(filename=root.joinpath(audio_name).with_suffix(".flac"))

In [ ]:
# Definición del nuevo dataset, para aplicarle posteriormente la variable definida
# en las celdas anteriores 'RFCXDataset'

data = pd.DataFrame({
    "recording_id": [path.stem for path in Path(TEST_AUDIO_ROOT).glob("*.flac")],
})
data["species_id"] = [[] for _ in range(len(data))]

print(data.shape)
data["duration"] = data["recording_id"].apply(get_duration)

# 6. Inferencia

In [ ]:
# En este último paso, utilizando el paquete 'resnest' y aplicando 'RFCXDataset' a los datos se obtiene,
# para cada 'recording_id', la probabilidad de cada una de las 23 especies de aparecer en el audio en
# concreto, que viene a ser el 'dataset' requerido por la competición

# Se definen los parámetros necesarios

TEST_BATCH_SIZE = 40
TEST_NUM_WORKERS = 2

In [ ]:
# Aplicamos las funciones al 'dataset' data

test_data = RFCXDataset(data=data, sr=SR)

# Utilizamos PyTorch para cargar los datos, iterando sobre 'test_data'

test_loader = DataLoader(test_data, batch_size=TEST_BATCH_SIZE, num_workers=TEST_NUM_WORKERS)

Estas 2 celdas de acontinuación son las que generan los resultados, utilizando el 'dataset' [RFCX Species Detection Public Checkpoints](https://www.kaggle.com/kneroma/kkiller-rfcx-species-detection-public-checkpoints) de Kkiller para evaluar y guardando las predicciones obtenidas en el 'dataset' "preds"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net = resnest50(pretrained=True).to(device)
n_features = net.fc.in_features
net.fc = torch.nn.Linear(n_features, NUM_CLASSES)
net = net.to(device)
net.load_state_dict(torch.load("../input/kkiller-rfcx-species-detection-public-checkpoints/rfcx_resnest50.pth", map_location=device))
net = net.eval()
net

In [ ]:
preds = []
net.eval()
with torch.no_grad():
    for (xb, yb) in  tqdm(test_loader):
        xb, yb = xb.to(device), yb.to(device)
        o = net(xb)
        o = torch.sigmoid(o) 
        preds.append(o.detach().cpu().numpy())
preds = np.vstack(preds)
preds.shape

Finalmente, una vez obtenido "preds", definimos el 'dataset' que subiremos a la competición, dándole el formato requerido según las instrucciones de Rainforest Connection, y concluyendo así el Notebook!

In [ ]:
sub = pd.DataFrame(preds, columns=[f"s{i}" for i in range(24)])
sub["recording_id"] = data["recording_id"].values[:len(sub)]
sub = sub[["recording_id"] + [f"s{i}" for i in range(24)]]
print(sub.shape)
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)